# HyLoRADA: Hybrid Low-Rank Adaptation with Landmark Memory

**Components**: rsLoRA + DoRA + LandmarkLoRA + PositionBias

| Component | Description | Params |
|-----------|-------------|--------|
| rsLoRA | α/√r scaling | 0 |
| DoRA | Magnitude normalization | ~86K |
| LandmarkLoRA | Context summaries (Novel) | ~14K |
| PositionBias | Position awareness | 64 |

In [ ]:
# Setup
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Clone repo (Kaggle)
import os
if os.path.exists('hylorada'):
    %cd hylorada
    !git pull
else:
    !git clone https://github.com/SadiaTabassum1216/hylorada.git
    %cd hylorada

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate tqdm

In [ ]:
# Check GPU
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 1. Quick Demo - HyLoRADA Components

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from hylorada import HyLoRADAConfig, HyLoRADAModel, LandmarkLoRA

# Test LandmarkLoRA (Novel component)
lm = LandmarkLoRA(hidden_size=896, num_landmarks=8)
x = torch.randn(1, 64, 896)
y = lm(x)
print(f"LandmarkLoRA: {x.shape} -> {y.shape}")
print(f"LandmarkLoRA params: {sum(p.numel() for p in lm.parameters()):,}")

In [ ]:
# Load base model
model_name = "Qwen/Qwen2-0.5B"
base_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# HyLoRADA Config
config = HyLoRADAConfig(
    lora_rank=8,
    lora_alpha=16.0,
    use_dora_magnitude=True,    # DoRA
    landmark_enabled=True,       # LandmarkLoRA (Novel)
    num_landmarks=8,
    position_bias_enabled=True,
)

print("Components:", config.get_component_status())

In [ ]:
# Apply HyLoRADA
model = HyLoRADAModel(base_model, config)
model.print_trainable_params()

## 2. Run Benchmark

In [ ]:
# Quick benchmark (5 min)
!python run_benchmark.py --methods lora dora hylorada --epochs 1 --num_train 200

In [ ]:
# Full benchmark (longer)
# !python run_benchmark.py --methods lora dora hylorada --epochs 3 --num_train 1000

## 3. Long-Context Configuration

For sequences >2048 tokens, enable:
- S²-Attn for training efficiency
- Trainable embeddings (LongLoRA)
- RoPE scaling (YaRN)

In [ ]:
# Long-context config
long_config = HyLoRADAConfig(
    lora_rank=8,
    use_dora_magnitude=True,
    landmark_enabled=True,
    # Long-context extensions
    s2_attn_enabled=True,
    s2_group_size=2048,
    train_embeddings=True,
    train_norms=True,
    rope_scaling_type="linear",
    rope_scaling_factor=4.0,
)

print("Long-context components:", long_config.get_component_status())